# K-means Algorithm

**Name**: ZHU GUANGYU  
**Student ID**: 20165953  
**Github Repo**: [assignment03]()  

---

## Clustering

the goal of *clustering* is to group or partition the vectors into *k* groups or clusters, with the vectors in each group close to each other.

the best clustering:
- can find the best slustering, if the representatives are fixed;
- can find the vest representatives, if the clustering is fixed.

We use a single number to judge a choice of clustering, along with a choice of the group. We define:

$$
J^{clust}=(||x_{1}-z_{c_{1}}||^{2}+ \cdots +||x_{N}-z_{c_{N}}||^{2})/N
$$

Here $x_{N}$ is vector, $z_{c_{N}}$ is correspond representatives. We call this value *energy* or *cost*.

### When representatives fixed

We assign each date vector $x_{i}$ to its nearest representatives. Since the representatives are fixed, we actually re-grouped vectors into different partitions. We have:
$$ ||x_{i}-z_{c_{i}}|| = \min_{j=1,\cdots\,k}||x_{i}-z_{c_{i}}|| $$ 

This gives us the minimum $J^{clust}$.

### When group assignment fixed

This means the element vectors of each group are fixed. We need to find the group representatives to minimize our cost $J^{clust}$.

Simply, choose the average of the vectros in its group:
$$ z_{j} = (1/|G_{j}|)\sum_{i\in G_{j}}x_{i}$$

since this makes the sum of distance between points and its representative minimum.

## *k*-means Algorithm

Previous two methods can help us get the best clustering. But the two methods are depend on each other. To solve this problem, we can use *k-means algorithm*.

*k-means algorithm*'s idea is simple. We repeatedly alternate between updating the group assignments, then updating the representatives. In each iteration we get a better $J^{clust}$ until the step does not change the choice.

Have to be aware of is k-means algorithm **cannot** guarantee that the partition it finds minimizes $J^{clust}$. Commonly, we run it several times with different initial representatives, and choose the one with the smallest cost.

There is another problem is to determin the optimal number of clusters (here is the *k*).  
If you have given number of clusters, that's fine. If you don't, there are few methods can help us:
- [Elbow method](https://en.wikipedia.org/wiki/Elbow_method_%28clustering%29 )
- [The silhouette method](https://en.wikipedia.org/wiki/Silhouette_%28clustering%29)
- [Gap statistic](http://web.stanford.edu/~hastie/Papers/gap.pdf)